In [21]:
import pandas as pd
import warnings
import requests
import os
import subprocess
import whisper
import shutil

warnings.filterwarnings('ignore')

In [11]:
df = pd.read_csv('../raw_data/data_rotulada_final_2L.csv')

In [17]:
df['video_id'].nunique()

3451

# Filtragem dos vídeos disponíveis

O primeiro passo que precisamos fazer é filtrar os vídeos que estão no nosso dataset para sabermos quais ainda estão disponíveis no Youtube.

In [ ]:
def check_video_availability(video_id):
    url = f"https://www.youtube.com/oembed?url=https://www.youtube.com/watch?v={video_id}&format=json"
    response = requests.get(url)
    
    return response.status_code == 200

In [13]:
def save_progress(valid_videos, invalid_videos):
    """Salva os vídeos válidos e inválidos em arquivos CSV separados."""
    valid_videos.to_csv("../filtrated_data_log/valid_videos.csv", index=False)
    invalid_videos.to_csv("../filtrated_data_log/invalid_videos.csv", index=False)

In [14]:
def load_progress():
    """Carrega o progresso de vídeos válidos e inválidos (se houver)."""
    try:
        valid_videos = pd.read_csv("../filtrated_data_log/valid_videos.csv")
    except FileNotFoundError:
        valid_videos = pd.DataFrame(columns=["video_id"])
    
    try:
        invalid_videos = pd.read_csv("../filtrated_data_log/invalid_videos.csv")
    except FileNotFoundError:
        invalid_videos = pd.DataFrame(columns=["video_id"])
    
    return valid_videos, invalid_videos

In [ ]:
valid_videos, invalid_videos = load_progress()

In [ ]:
for video_id in df['video_id'].drop_duplicates():
    if video_id not in valid_videos["video_id"].values and video_id not in invalid_videos["video_id"].values:
        status = check_video_availability(video_id)
        
        if status is not None:
            new_row = pd.DataFrame({"video_id": [video_id]})
            if status:
                valid_videos = pd.concat([valid_videos, new_row], ignore_index=True)
                print(f"O vídeo {video_id} está disponível.")
            else:
                invalid_videos = pd.concat([invalid_videos, new_row], ignore_index=True)
                print(f"O vídeo {video_id} foi removido ou está indisponível.")
        
        save_progress(valid_videos, invalid_videos)

# Filtrar todos os dados dos vídeos disponíveis

O objetivo desse bloco é verificar as informações dos vídeos que estão disponíveis, para isso, eu deixei o dataset com os dados dos vídeos, como título, descrição e etc. Com isso, conseguimos fazer análises sobre a quantidade de vídeos favoráveis/contrários entre outras.

In [24]:
# df_dados = pd.read_csv('../raw_data/videos_info_clean.csv')
df_filtro = pd.read_csv('../filtrated_data_log/valid_videos.csv')

In [25]:
df['video_id'] = df['video_id'].astype(str)
df_filtro['video_id'] = df_filtro['video_id'].astype(str)

In [26]:
dados_filtrados = df[df['video_id'].isin(df_filtro['video_id'])]

In [28]:
dados_filtrados.to_csv('../filtrated_data_log/videos_filtrados.csv', index=False)
dados_filtrados.head()

,Unnamed: 0,video_id,comment_id,author,author_profile_image_url,author_channel_url,author_channel_id,text,published_at,updated_at,...,viewer_rating,can_rate,is_reply,parent_id,channel_id,language,language_langdetect,label_llama3.1,label_roberta,label
0,0,EMkU6IJSS4k,UgzodyqhxQCoKjflca94AaABAg,@neto.46,https://yt3.ggpht.com/ytc/AIdro_nV3tOuBE_fgYKR...,http://www.youtube.com/@neto.46,UC2ul86IJ-WnSHEQy9R8pR7Q,pessoas que acreditam nesses jogos são trouxas...,2024-06-30 23:04:00+00:00,2024-06-30T23:04:00Z,...,none,True,False,NaN,UCaGmdJSSiR7fkh2A-c6emsA,pt,NaN,0,0,0
1,2,EMkU6IJSS4k,UgxT8qjVDcUBKSumdoh4AaABAg,@eduardomacapol5893,https://yt3.ggpht.com/ytc/AIdro_nta6z5KQYI-gru...,http://www.youtube.com/@eduardomacapol5893,UCuWHlFoFOGb0c08FjGhRScg,Mais prejudicial do que esses jogos é a midia ...,2024-06-30 22:38:47+00:00,2024-06-30T22:38:47Z,...,none,True,False,NaN,UCaGmdJSSiR7fkh2A-c6emsA,pt,NaN,0,0,0
2,3,EMkU6IJSS4k,UgwcgM5w4EHgT3be5Yx4AaABAg,@josealvaro-bc7zv,https://yt3.ggpht.com/ytc/AIdro_nQWtDjnkGaIYmO...,http://www.youtube.com/@josealvaro-bc7zv,UCxeyqepx5lCqCS2XYJVl_hg,",,,, prender essa gente toda ao dizer quem são...",2024-06-30 22:28:55+00:00,2024-06-30T22:28:55Z,...,none,True,False,NaN,UCaGmdJSSiR7fkh2A-c6emsA,pt,NaN,0,0,0
3,6,EMkU6IJSS4k,UgyoIIKIadGYM3nvC5F4AaABAg,@fernandojunior8700,https://yt3.ggpht.com/ytc/AIdro_knA9dAYsVClGS8...,http://www.youtube.com/@fernandojunior8700,UCjCSM9V-ETEbMfijKM7KccA,"Joga quem quer, a pessoa que divulga não tem n...",2024-06-30 22:00:50+00:00,2024-06-30T22:00:50Z,...,none,True,False,NaN,UCaGmdJSSiR7fkh2A-c6emsA,pt,NaN,0,0,0
4,8,EMkU6IJSS4k,UgyMjIrRXF6z1rAhuF94AaABAg,@vagnercarvalhosaraiva4426,https://yt3.ggpht.com/ytc/AIdro_mvsJmN2LFaOFOV...,http://www.youtube.com/@vagnercarvalhosaraiva4426,UCec0yjiQKsz31_nw8MOOA5A,assim q lula ladrão estiver taxando o tigrinho...,2024-06-30 21:30:05+00:00,2024-06-30T21:30:05Z,...,none,True,False,NaN,UCaGmdJSSiR7fkh2A-c6emsA,pt,NaN,0,0,0


# Fazendo o download dos vídeos

In [32]:
# Configurações
audio_folder = "../audios/"
log_file = "../download_log/download_progress.log"
csv_file = "../filtrated_data_log/valid_videos.csv"

In [ ]:
def load_progress():
    if os.path.exists(log_file):
        with open(log_file, "r") as f:
            return set(f.read().splitlines())
    return set()

def save_progress(video_id):
    with open(log_file, "a") as f:
        f.write(video_id + "\n")

In [ ]:
df = pd.read_csv(csv_file)
video_ids = df["video_id"].astype(str).tolist()

In [35]:
completed_videos = load_progress()

In [ ]:
for video_id in video_ids:
    audio_path = os.path.join(audio_folder, f"{video_id}.mp3")

    # Se já foi baixado, pula
    if video_id in completed_videos or os.path.exists(audio_path):
        print(f"Áudio já baixado: {audio_path}")
        continue

    url = f"https://www.youtube.com/watch?v={video_id}"
    print(f"Baixando áudio de {url}...")

    try:
        result = subprocess.run(
            ["yt-dlp", "-f", "bestaudio", "--extract-audio", "--audio-format", "mp3",
             "-o", f"{audio_folder}/%(id)s.%(ext)s", url],
            check=True, text=True, capture_output=True
        )
        print(f"Download concluído: {audio_path}")
        save_progress(video_id)

    except subprocess.CalledProcessError as e:
        print(f"Erro ao baixar {video_id}: {e.stderr}")

print("Processo de download concluído!")

Baixando áudio de https://www.youtube.com/watch?v=EMkU6IJSS4k...
Download concluído: ../audios/EMkU6IJSS4k.mp3
Baixando áudio de https://www.youtube.com/watch?v=5fXght1EiwI...
Download concluído: ../audios/5fXght1EiwI.mp3
Baixando áudio de https://www.youtube.com/watch?v=Zgzrijrbex4...
Download concluído: ../audios/Zgzrijrbex4.mp3
Baixando áudio de https://www.youtube.com/watch?v=KkIc58ZvdSA...


# Transcrever os Áudios

In [ ]:
audio_folder = "../audios/"
transcription_folder = "../transcriptions"
log_file = "../download_log/progresso_transcricao.log"

In [14]:
df = pd.read_csv("../raw_data/data_rotulada_final_2L.csv")
video_ids = df["video_id"].drop_duplicates().tolist()

In [15]:
def load_progress():
    if os.path.exists(log_file):
        with open(log_file, "r") as f:
            return set(f.read().splitlines())
    return set()

def save_progress(video_id):
    with open(log_file, "a") as f:
        f.write(video_id + "\n")

In [16]:
completed_videos = load_progress()

In [17]:
modelo = whisper.load_model("small").to("cuda")

In [ ]:
for video_id in df['video_id'].drop_duplicates():
    audio_path = os.path.join(audio_folder, f"{video_id}.mp3")
    output_text = os.path.join(transcription_folder, f"{video_id}.txt")
    zip_audio_base = os.path.join(audio_folder, video_id)  # Sem extensão

    # Se já foi transcrito, pula
    if video_id in completed_videos or os.path.exists(output_text):
        print(f"Transcrição já feita para {video_id}")
        continue

    if not os.path.exists(audio_path):
        print(f"Áudio não encontrado para {video_id}, pulando...")
        continue

    print(f"Transcrevendo {audio_path}...")

    try:
        result = modelo.transcribe(audio_path, language="portuguese")

        with open(output_text, "w", encoding="utf-8") as f:
            f.write(result["text"])

        save_progress(video_id)
        print(f"Transcrição salva: {output_text}")

        # Compacta apenas o arquivo MP3
        shutil.make_archive(zip_audio_base, 'zip', audio_folder, f"{video_id}.mp3")
        print(f"Áudio compactado: {zip_audio_base}.zip")
        
        os.remove(audio_path)
        print(f"Áudio excluído: {audio_path}")

    except Exception as e:
        print(f"Erro ao transcrever {video_id}: {str(e)}")

print("Processo de transcrição concluído!")